In [7]:
import re 
import sys 
import json
from io import StringIO
from abc import ABC, abstractmethod
from enum import Enum
from typing import Dict, List, Any, Optional
from dataclasses import dataclass

from langchain_ollama import (
  OllamaLLM,
  ChatOllama,
  OllamaEmbeddings
)
from langchain_core.messages import (
  BaseMessage,
  AIMessage,
  HumanMessage  
)
from langchain_core.prompts import (
  ChatPromptTemplate,
  MessagesPlaceholder  
)


GEMMA = "gemma3:1b"             # ollama gemma3:1b
DEEPSEEK = "deepseek-r1:1.5b"   # ollama deepseek-r1:1.5b

In [11]:
DEFAULT_SYSTEM_PROMPT = """ 
Eres un asistente especializado capaz de responder preguntas sencillas
"""

class LLM(ABC):
  def __init__(self, model:str, api_key:str="", temperature:float=0.7):
    self.model = model
    self.api_key = api_key 
    self.temperature = temperature
  
  @abstractmethod
  def __call__(self, query:str) -> str:
    pass 

class Ollama(LLM):
  def __init__(self, model:str, api_key:str = "", temperature:float = 0.7, system_prompt:str = ""):
    super().__init__(model, api_key, temperature)
    
    try:
      self.llm = OllamaLLM(self.model, self.temperature)
    except Exception:
      self.llm = OllamaLLM(model=GEMMA, temperature=0.7)
    
    self.system_prompt:str = system_prompt if len(system_prompt) > 0 else DEFAULT_SYSTEM_PROMPT
    self.memory:List = []
    
    self.chat_prompt = ChatPromptTemplate.from_messages(
      [
        ('system', f'{self.system_prompt}'),
        MessagesPlaceholder(variable_name='memory'),
        ('human', '{query}')
      ]
    )
    self.chain = self.chat_prompt | self.llm  
  
  def __call__(self, query:str) -> str:
    response = self.chain.invoke(
      {
        "query": HumanMessage(query),
        "memory": self.memory
      }
    )
    self.memory.append(HumanMessage(content=query))
    self.memory.append(AIMessage(content=response))
    
    return response

  def reset_historial(self) -> None:
    self.memory:List = [] 

In [12]:
queries = [
  "¿Qué es un número complejo?",
  "¿Qué tipo de operaciones tiene?",
  "¿Dime un proyecto que utilice esto?"
]

prompt = """ 
Eres un asistente de inteligencia artificial capaz de dar respuestas 
razonando antes con la conversion tenida con el usuario
Lenguaje de respuesta: Español
"""

chatbot = Ollama(prompt)

r = []
for query in queries:
  rt = chatbot(query)
  r.append(rt)
  print(rt)
  print("\n\n===============\n\n")  

Un número complejo es un número que puede ser escrito como un producto de dos números, uno de los cuales es un número complejo.  En otras palabras, es un número que tiene una parte real y una parte imaginaria.

¿Hay algo más en lo que pueda ayudarte?





AI: Sí, hay varias operaciones que se pueden realizar con números complejos:

*   **Elevar al cuadrado:**  (z^2)
*   **Elevar al cubo:** (z^3)
*   **Elevar al cuarto:** (z^4)
*   **Elevar al quinto:** (z^5)
*   **Multiplicación:** (z * w)
*   **División:** (z / w)
*   **Suma:** (z + w)
*   **Resta:** (z - w)
*   **Producto:** (z * w)
*   **Producto interno:** (z * w)
*   **Producto externo:** (w * z)

¿Te gustaría saber más sobre alguna de estas operaciones o sobre cómo se relacionan con los números reales?




AI: ¡Claro! Un proyecto interesante que podrías usar números complejos es la **simulación de una onda sinusoidal**.

Aquí te explico cómo:

1.  **Representación de la onda:** Puedes representar una onda sinusoidal (como una ond

## Test: Rol y Modificaciones en Temperatura

In [16]:
def test_languages(prompt:str, temperature:float, query:str, n:int=20) -> List[str]:
  chatbot = Ollama(
    model="",
    system_prompt=prompt,
    temperature=temperature
  )
  result = []
  for _ in range(n):
    response = chatbot(query)
    result.append(response)
    chatbot.reset_historial()
  
  return result

In [17]:
prompt = """ 
Eres un asistente de inteligencia artificial capaz de dar respuestas sencillas y cortas. 
Lenguaje de respuesta: Español.
"""
temperature = 0.8
query = """ 
¿Qué es el Machine Learning? ¿Qué técnicas presenta?
"""

result = test_languages(prompt, temperature, query)

In [19]:
unique_result = set(result)
print(f"Resultados únicos: {len(unique_result)}")

for i,r in enumerate(result, start=1):
  print(f"============= {i} =============")
  print(r)


Resultados únicos: 20
============= 1 =============
¡Hola! El Machine Learning (aprendizaje automático) es un campo de la inteligencia artificial que se centra en enseñar a los sistemas para que puedan aprender y mejorar sin ser programados explícitamente. En lugar de programar cada paso, se alimenta al modelo con datos y el sistema aprende patrones y reglas a partir de estos datos.

**¿Qué técnicas presenta?**

Existen varias técnicas clave utilizadas en el Machine Learning:

*   **Aprendizaje Supervisado:** Se alimenta al modelo con datos etiquetados (es decir, datos que ya tienen la respuesta correcta). El modelo aprende a predecir la respuesta correcta para nuevos datos sin etiquetar. Ejemplos: regresión, clasificación.
*   **Aprendizaje No Supervisado:** Se alimenta al modelo con datos sin etiquetar. El modelo debe descubrir patrones y estructuras en los datos por sí mismo. Ejemplos: clustering, reducción de dimensionalidad.
*   **Aprendizaje por Refuerzo:** El modelo aprende a to